# The Sellar Problem

First, we have seen quick applications of WISDEM for to run a cost and scaling model.

Then, we used OpenMDAO to study the Betz limit and find the maximum power that can be captured by an idealized rotor. There, we declared partial derivatives and provided analytical solutions to those derivatives and allowed OpenMDAO to optimize our problem.

The Betz problem used just one `ActuatorDisc` component. In real applications of WISDEM, we will need to optimize with multiple components. This tutorial will show how to assemble multiple components in OpenMDAO. From there there, our next step will return to WISDEM to assemble a wind turbine in code.

## Multidisciplinary design optimization (MDO) terminology and XDSM diagrams

MDO has some key terms and visual diagram format called XDSM to describe optimization setups. There are couple resources to learn the basics that could be helpful. The entirety of these articles are a good reference, but there are a few key sections that have information we'll use in this demo:

- [Problem formulation section of multidisciplinary design optimization on Wikipedia](https://en.wikipedia.org/wiki/Multidisciplinary_design_optimization#Problem_formulation): Read the definitions for *design variables*, *constraints*, *objectives* and *models*.

- [Lambe and Martins: Extensions to the Design Strcuture Matrix for the Description of Multidisciplinary Desgn, Analysis, and Optimation Processes](http://mdolab.engin.umich.edu/content/extensions-design-structure-matrix): Read section 2 "Terminology and Notation" for further explanation of *design variables*, *discipline analysis*, *response variables*, *target variables* and *coupling variables*. Read section 4 about XDSM diagrams that dsecribe MDO processes.

This tutorial is based on the OpenMDAO documentation at [Sellar - A Two-Discipline Problem with a Nonlinear Solver](http://openmdao.org/twodocs/versions/latest/basic_guide/sellar.html). The aim of this tutorial is to summarize the key points you'll use to create basic WISDEM models.

## Problem setup

The Sellar problem are a couple of components (what Wikipedia calls models) that are simple equations. There is an objective to optimize and a couple of constraints to follow.

![Sellar XDSM](img/sellar_xdsm.png)

First we need to import OpenMDAO

In [1]:
import openmdao.api as om
import numpy as np

Let's build *Discipline 1* first. Remember that we decarled and provided analytical derivatives in the Betz limit example. This time, we will declare partial derivatives. However, we will ask OpenMDAO to use the finite difference method to approximate the partial derivatives. On the XDSM diagram, notice the parallelograms connected to *Discipline 1* by thick grey lines. These are variables pertaining to the  *Discipline 1* component.

- \\(\mathbf{z}\\): An input. Since the components \\(z_1, z_2\\) can form a vector, so we call the variable `z` in the code and initialize it to \\((0, 0)\\) with `np.zeros(2)`. Note that components of \\(\mathbf{z}\\) are found in 3 of the white \\(\mathbf{z}\\) parallelograms connected to multiple components and the objective, so this is a globabl design variable.

- \\(x\\): An input. A local design variable for Discipline 1. Since it isn't a vector, we just initialize it as a float.

- \\(y_2\\): An input. This is a coupling variable coming from an output on *Discipline 2*

- \\(y_1\\): An output. This is acoupling variable going to an input on *Discipline 2*

Finally `self.declare_partials('*', '*', method='fd')` tell OpenMDAO to compute the partial derivative of any variable with respect to any other variable as a finite difference approximation.

In [15]:
class SellarDis1(om.ExplicitComponent):
    """
    Component containing Discipline 1 -- no derivatives version.
    """

    def setup(self):

        # Global Design Variable
        self.add_input('z', val=np.zeros(2))

        # Local Design Variable
        self.add_input('x', val=0.)

        # Coupling parameter
        self.add_input('y2', val=1.0)

        # Coupling output
        self.add_output('y1', val=1.0)

        # Finite difference all partials.
        self.declare_partials('*', '*', method='fd')


    def compute(self, inputs, outputs):
        """
        Evaluates the equation
        y1 = z1**2 + z2 + x1 - 0.2*y2
        """
        z1 = inputs['z'][0]
        z2 = inputs['z'][1]
        x1 = inputs['x']
        y2 = inputs['y2']

        outputs['y1'] = z1**2 + z2 + x1 - 0.2*y2

Now, *Discipline 2*.

- \\(\mathbf{z}\\): An input comprised of \\(z_1, z_2\\).

- \\(y_2\\): An output. This is a coupling variable going to an input on *Discipline 1*

- \\(y_1\\): An input. This is a coupling variable coming from an output on *Discipline 1*

In [16]:
class SellarDis2(om.ExplicitComponent):
    """
    Component containing Discipline 2 -- no derivatives version.
    """

    def setup(self):
        # Global Design Variable
        self.add_input('z', val=np.zeros(2))

        # Coupling parameter
        self.add_input('y1', val=1.0)

        # Coupling output
        self.add_output('y2', val=1.0)

        # Finite difference all partials.
        self.declare_partials('*', '*', method='fd')


    def compute(self, inputs, outputs):
        """
        Evaluates the equation
        y2 = y1**(.5) + z1 + z2
        """

        z1 = inputs['z'][0]
        z2 = inputs['z'][1]
        y1 = inputs['y1']

        # Note: this may cause some issues. However, y1 is constrained to be
        # above 3.16, so lets just let it converge, and the optimizer will
        # throw it out
        if y1.real < 0.0:
            y1 *= -1

        outputs['y2'] = y1**.5 + z1 + z2

So those are the components. So far in our lessons, we have used `om.ExplicitComponent` for our models. Now, we need more classes to group multiple components (our Discipline 1 and 2) with an objective function, constraints and connect all the variables correctly.. To do this we'll use more OpenMDAO classes. There is [more detail in the OpenMDAO docuemntation](http://openmdao.org/twodocs/versions/latest/basic_guide/sellar.html), but here are the highlights:

- `Group`: The OpenMDAO `om.Group` class allows you to cluster models in hierarchies. We can put multiple components in groups. We can also put other groups in groups. By creating groups you can create assemblies of models.

- `ExecComp`: This is a great solution for making an equation for a constraint or objective without building out an entire component.

- `NonlinearBlockGS`: This is one of the many solvers in OpenMDAO. It is a non-linear solver that we'll use to optimize our disciplines as a subgroup before we optimize the rest of the sytem around them.

Here are few of the highlights from the source code below:

### Promoting variables of the same name connects them

When you promote an output from a component, it is connected to the inputs of the same name within the same subsystem.

### Design variables added as outputs from an `IndepVarComp` component:

```python
indeps = self.add_subsystem('indeps', om.IndepVarComp(), promotes=['*'])
indeps.add_output('x', 1.0)
indeps.add_output('z', np.array([5.0, 2.0]))
```

### A cycle that interconnects our discipline components

For solving the Sellar system, it is useful to iteratively converge the discipline components first, and then calcualte the rest of the model. We do this by making a subgroup that holds our discipline components and giving them a solver.

```python
cycle = self.add_subsystem('cycle', om.Group(), promotes=['*'])
cycle.add_subsystem('d1', SellarDis1(), promotes_inputs=['x', 'z', 'y2'],
                    promotes_outputs=['y1'])
cycle.add_subsystem('d2', SellarDis2(), promotes_inputs=['z', 'y1'],
                    promotes_outputs=['y2'])
cycle.nonlinear_solver = om.NonlinearBlockGS()
```

### `ExecComp` is handy for turning smple equations into components

Look at the following snippet:

```python
self.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])
```

This quickly creates a component for constraint 2. The other constraint and the objective are coded similarly.

### Here's the code for the class:

In [17]:
class SellarMDA(om.Group):
    """
    Group containing the Sellar MDA.
    """

    def setup(self):
        indeps = self.add_subsystem('indeps', om.IndepVarComp(), promotes=['*'])
        indeps.add_output('x', 1.0)
        indeps.add_output('z', np.array([5.0, 2.0]))

        cycle = self.add_subsystem('cycle', om.Group(), promotes=['*'])
        cycle.add_subsystem('d1', SellarDis1(), promotes_inputs=['x', 'z', 'y2'],
                            promotes_outputs=['y1'])
        cycle.add_subsystem('d2', SellarDis2(), promotes_inputs=['z', 'y1'],
                            promotes_outputs=['y2'])

        # Nonlinear Block Gauss Seidel is a gradient free solver
        cycle.nonlinear_solver = om.NonlinearBlockGS()

        self.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                                  z=np.array([0.0, 0.0]), x=0.0),
                           promotes=['x', 'z', 'y1', 'y2', 'obj'])

        self.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
        self.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

## Let's optimize our system!

Even though we have all the pieces in a `Group`, we still need to pull them apart and put them into a `Problem` to be solved.

Here, we add some ranges for our design variables. These are from our `SellarMDA` group in the `indeps` subsystem.

```python
prob.model.add_design_var('x', lower=0, upper=10)
prob.model.add_design_var('z', lower=0, upper=10)
```

Then we have our objective `obj`. This is from our `SellarMDA` group in the `obj_cmp` subsystem.

```python
prob.model.add_objective('obj')
```

Finally we have our constraints `con1` and `con2`. This is from our `SellarMDA` group in the `con_cmp1` and `con_cmp2` subsystems.

```python
prob.model.add_constraint('con1', upper=0)
prob.model.add_constraint('con2', upper=0)
```

In [19]:
prob = om.Problem()
prob.model = SellarMDA()

prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
prob.driver.options['tol'] = 1e-8

prob.model.add_design_var('x', lower=0, upper=10)
prob.model.add_design_var('z', lower=0, upper=10)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0)
prob.model.add_constraint('con2', upper=0)

prob.setup()
prob.set_solver_print(level=0)

# Ask OpenMDAO to finite-difference across the model to compute the gradients for the optimizer
prob.model.approx_totals()

prob.run_driver()

print('minimum found at')
print(prob['x'][0])
print(prob['z'])

print('minumum objective')
print(prob['obj'][0])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3.183393951729169
            Iterations: 6
            Function evaluations: 6
            Gradient evaluations: 6
Optimization Complete
-----------------------------------
minimum found at
0.0
[1.97763888e+00 8.83056605e-15]
minumum objective
3.183393951729169
